In [3]:
"""
SILVER LAYER EXPLORATORY ANALYSIS
Understanding data structure and analytical capabilities
"""

print("="*80)
print("🔍 SILVER LAYER EXPLORATORY ANALYSIS")
print("="*80)

from pyspark.sql.functions import *
from datetime import datetime
import matplotlib.pyplot as plt
import pandas as pd

print(f"Analysis started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# ============================================================================
# 1. UNDERSTAND EACH TABLE STRUCTURE
# ============================================================================
print("\n1️⃣ TABLE STRUCTURE ANALYSIS")
print("="*80)

# List all active tables (exclude backups)
active_tables = [
    # Core fact tables
    "silver_taxi_clean",
    "silver_taxi_daily",
    "silver_air_quality_daily",
    
    # Dimension tables
    "dim_date_clean",
    "dim_zones",
    
    # Economic data
    "silver_fx_clean",
    "silver_gdp_clean",
    
    # Detailed pollutant data
    "silver_pm25_std",
    "silver_no2_std",
    "silver_o3_std",
    "silver_pm10_std"
]

print(f"Analyzing {len(active_tables)} active tables...")

# Dictionary to store date info for later analysis
date_info = {}

for table_name in active_tables:
    print(f"\n📊 {table_name}:")
    try:
        df = spark.table(table_name)
        
        # Basic info
        row_count = df.count()
        print(f"   Rows: {row_count:,}")
        print(f"   Columns: {len(df.columns)}")
        
        # Show column names and sample data types
        print(f"   First 5 columns: {df.columns[:5]}")
        
        # Date columns
        date_cols = [c for c in df.columns if "date" in c.lower() or "time" in c.lower()]
        if date_cols:
            print(f"   Date columns: {date_cols}")
            # Store date info for later analysis
            for date_col in date_cols[:1]:  # Check first date column
                try:
                    dates = df.select(min(date_col), max(date_col)).collect()[0]
                    min_date = dates[0]
                    max_date = dates[1]
                    print(f"   Date range: {min_date} to {max_date}")
                    
                    # Store for later use
                    date_info[table_name] = {
                        "min_date": min_date,
                        "max_date": max_date,
                        "date_column": date_col
                    }
                except Exception as e:
                    print(f"   Error getting date range: {str(e)[:100]}")
        
        # Numeric columns
        numeric_cols = []
        for col_name in df.columns:
            try:
                # Try to check if numeric
                sample_df = df.select(col_name).limit(1)
                sample_row = sample_df.collect()[0]
                sample_value = sample_row[0]
                if isinstance(sample_value, (int, float)):
                    numeric_cols.append(col_name)
            except:
                pass
        
        if numeric_cols:
            print(f"   Numeric columns: {len(numeric_cols)} (e.g., {numeric_cols[:3]})")
            
        # Show small sample
        print(f"   Sample data:")
        df.limit(3).show(vertical=True, truncate=50)
        
    except Exception as e:
        print(f"   ❌ Error: {str(e)[:100]}")

# ============================================================================
# 2. CHECK DATE ALIGNMENT BETWEEN TABLES (FIXED VERSION)
# ============================================================================
print("\n" + "="*80)
print("2️⃣ DATE ALIGNMENT ANALYSIS")
print("="*80)

# Find overlapping periods (FIXED VERSION)
print("\n🔗 POTENTIAL JOIN POINTS:")

# Check taxi vs air quality
if "silver_taxi_daily" in date_info and "silver_air_quality_daily" in date_info:
    taxi_info = date_info["silver_taxi_daily"]
    air_info = date_info["silver_air_quality_daily"]
    
    # Extract dates as Python datetime objects
    try:
        # Convert to string for comparison if needed
        taxi_start = str(taxi_info["min_date"])
        taxi_end = str(taxi_info["max_date"])
        air_start = str(air_info["min_date"])
        air_end = str(air_info["max_date"])
        
        print(f"   Taxi dates: {taxi_start} to {taxi_end}")
        print(f"   Air dates: {air_start} to {air_end}")
        
        # Calculate overlap using SQL instead
        try:
            taxi_dates = spark.table("silver_taxi_daily").select("pickup_date").distinct()
            air_dates = spark.table("silver_air_quality_daily").select("measurement_date").distinct()
            
            overlapping_dates = taxi_dates.join(
                air_dates, 
                taxi_dates.pickup_date == air_dates.measurement_date
            )
            overlap_count = overlapping_dates.count()
            
            if overlap_count > 0:
                # Get actual overlap dates
                overlap_min_max = overlapping_dates.select(
                    min("pickup_date").alias("min_overlap"),
                    max("pickup_date").alias("max_overlap")
                ).collect()[0]
                
                print(f"   ✅ Taxi ↔ Air Quality overlap: {overlap_min_max['min_overlap']} to {overlap_min_max['max_overlap']}")
                print(f"   Days with BOTH taxi and air data: {overlap_count}")
                
                print(f"   Sample overlapping dates:")
                overlapping_dates.limit(5).show()
            else:
                print(f"   ⚠️  NO DATE OVERLAP between taxi and air quality data")
                
        except Exception as e:
            print(f"   Error analyzing overlap: {str(e)[:100]}")
            
    except Exception as e:
        print(f"   Error processing dates: {str(e)[:100]}")

# Check FX overlap with taxi
print("\n💱 FX ↔ Taxi Date Alignment:")
if "silver_taxi_daily" in date_info and "silver_fx_clean" in date_info:
    fx_info = date_info["silver_fx_clean"]
    taxi_info = date_info["silver_taxi_daily"]
    
    try:
        taxi_start = str(taxi_info["min_date"])
        taxi_end = str(taxi_info["max_date"])
        fx_start = str(fx_info["min_date"])
        fx_end = str(fx_info["max_date"])
        
        print(f"   Taxi period: {taxi_start} to {taxi_end}")
        print(f"   FX period: {fx_start} to {fx_end}")
        
        # Check specific coverage
        taxi_days = spark.table("silver_taxi_daily").select("pickup_date").distinct()
        fx_dates = spark.table("silver_fx_clean").select("date").distinct()
        
        fx_coverage = taxi_days.join(fx_dates, taxi_days.pickup_date == fx_dates.date).count()
        total_taxi_days = taxi_days.count()
        
        if fx_coverage > 0:
            print(f"   ✅ Taxi days with FX rates: {fx_coverage}/{total_taxi_days} ({fx_coverage/total_taxi_days*100:.1f}%)")
            
            # Show sample with FX rates
            print(f"   Sample taxi days with FX rates:")
            taxi_with_fx = spark.table("silver_taxi_daily").join(
                spark.table("silver_fx_clean"),
                col("pickup_date") == col("date"),
                "inner"
            ).limit(5)
            
            taxi_with_fx.select("pickup_date", "trip_count", "total_revenue_usd", "usd_eur_rate").show()
        else:
            print(f"   ⚠️  No FX rates available for taxi dates")
            
    except Exception as e:
        print(f"   Error checking FX coverage: {str(e)[:100]}")

# ============================================================================
# 3. ANALYTICAL QUESTION EXPLORATION
# ============================================================================
print("\n" + "="*80)
print("3️⃣ ANALYTICAL CAPABILITIES - REALITY CHECK")
print("="*80)

print("\n🔍 REALISTIC ANALYSIS POSSIBILITIES:")

# 1. Taxi patterns analysis
print("\n🚕 TAXI PATTERNS ANALYSIS:")
try:
    taxi_daily = spark.table("silver_taxi_daily")
    
    # Get Jan 2024 data (main taxi period)
    jan_2024 = taxi_daily.filter(col("pickup_date").between("2024-01-01", "2024-01-31"))
    jan_days = jan_2024.count()
    
    if jan_days > 0:
        jan_trips = jan_2024.select(sum("trip_count")).collect()[0][0] or 0
        jan_revenue = jan_2024.select(sum("total_revenue_usd")).collect()[0][0] or 0
        
        print(f"   January 2024: {jan_days} days, {jan_trips:,} trips, ${jan_revenue:,.2f} revenue")
        
        # Show daily pattern
        print(f"   Daily pattern (first 7 days):")
        jan_2024.orderBy("pickup_date").limit(7).show()
    else:
        print(f"   No January 2024 data found")
        
    # Overall stats
    total_trips = taxi_daily.select(sum("trip_count")).collect()[0][0] or 0
    total_revenue = taxi_daily.select(sum("total_revenue_usd")).collect()[0][0] or 0
    print(f"\n   Total in dataset: {total_trips:,} trips, ${total_revenue:,.2f} revenue")
    
except Exception as e:
    print(f"   Error: {str(e)[:100]}")

# 2. Air quality analysis
print("\n🌫️ AIR QUALITY ANALYSIS:")
try:
    air_daily = spark.table("silver_air_quality_daily")
    
    # Check data completeness
    pollutants = ['pm25', 'no2', 'o3', 'pm10']
    print(f"   Data completeness by pollutant:")
    
    for pollutant in pollutants:
        col_name = f'avg_{pollutant}'
        if col_name in air_daily.columns:
            has_data = air_daily.filter(col(col_name).isNotNull()).count()
            total = air_daily.count()
            pct = (has_data / total * 100) if total > 0 else 0
            print(f"   {col_name.upper():20} {has_data:>4}/{total} days ({pct:.1f}%)")
    
    # Check for Jan 2024 air quality
    jan_air = air_daily.filter(col("measurement_date").between("2024-01-01", "2024-01-31"))
    jan_air_days = jan_air.count()
    print(f"\n   January 2024 air quality days: {jan_air_days}")
    
    if jan_air_days > 0:
        print(f"   Sample Jan 2024 air quality data:")
        jan_air.select("measurement_date", "avg_pm25", "avg_no2", "avg_o3").limit(3).show()
        
except Exception as e:
    print(f"   Error: {str(e)[:100]}")

# 3. Combined analysis potential
print("\n🔗 COMBINED ANALYSIS POTENTIAL:")
try:
    # Find dates with BOTH taxi and air quality in Jan 2024
    jan_taxi_dates = spark.table("silver_taxi_daily").filter(
        col("pickup_date").between("2024-01-01", "2024-01-31")
    ).select("pickup_date").distinct()
    
    jan_air_dates = spark.table("silver_air_quality_daily").filter(
        col("measurement_date").between("2024-01-01", "2024-01-31")
    ).select("measurement_date").distinct()
    
    jan_overlap = jan_taxi_dates.join(
        jan_air_dates, 
        jan_taxi_dates.pickup_date == jan_air_dates.measurement_date
    ).count()
    
    print(f"   January 2024 days with BOTH taxi and air data: {jan_overlap}")
    
    if jan_overlap > 0:
        print(f"   ✅ CAN CORRELATE taxi trips vs pollution for {jan_overlap} days in Jan 2024")
        
        # Show example correlation
        combined = spark.table("silver_taxi_daily").alias("taxi").join(
            spark.table("silver_air_quality_daily").alias("air"),
            col("taxi.pickup_date") == col("air.measurement_date")
        ).filter(
            col("taxi.pickup_date").between("2024-01-01", "2024-01-31")
        ).select(
            col("taxi.pickup_date").alias("date"), 
            "trip_count",
            "total_revenue_usd",
            "avg_pm25",
            "avg_no2"
        ).limit(5)
        
        print(f"   Example correlation data:")
        combined.show()
    else:
        print(f"   ⚠️  No overlapping days in January 2024 for correlation")
        
except Exception as e:
    print(f"   Error: {str(e)[:100]}")

# ============================================================================
# 4. TEST JOIN CAPABILITIES
# ============================================================================
print("\n" + "="*80)
print("4️⃣ TESTING JOIN CAPABILITIES")
print("="*80)

# Test 1: Taxi + Air Quality Join
print("\n🔗 TEST 1: Taxi Daily + Air Quality Join")
try:
    taxi_daily = spark.table("silver_taxi_daily")
    air_daily = spark.table("silver_air_quality_daily")
    
    # Find overlapping dates
    join_result = taxi_daily.join(
        air_daily, 
        taxi_daily.pickup_date == air_daily.measurement_date,
        "inner"
    )
    
    join_count = join_result.count()
    if join_count > 0:
        print(f"   ✅ JOIN WORKS! Found {join_count:,} overlapping records")
        print(f"   Sample joined data (first 3 records):")
        join_result.select(
            "pickup_date", 
            "trip_count", 
            "total_revenue_usd",
            "avg_pm25",
            "avg_no2"
        ).limit(3).show()
    else:
        print(f"   ⚠️  No overlapping dates found for join")
        
except Exception as e:
    print(f"   ❌ Join error: {str(e)[:100]}")

# Test 2: Taxi + FX Join
print("\n💱 TEST 2: Taxi Daily + FX Rates Join")
try:
    taxi_daily = spark.table("silver_taxi_daily")
    fx = spark.table("silver_fx_clean")
    
    # Check FX date column name
    fx_date_col = None
    for col in fx.columns:
        if "date" in col.lower():
            fx_date_col = col
            break
    
    if fx_date_col:
        join_sample = taxi_daily.join(
            fx, 
            taxi_daily.pickup_date == fx[f"{fx_date_col}"],
            "left"
        ).limit(5)
        
        print(f"   Using FX date column: {fx_date_col}")
        print(f"   Sample with FX rates:")
        join_sample.select(
            "pickup_date", 
            "total_revenue_usd",
            fx_date_col,
            "usd_eur_rate"
        ).show()
        
        # Check how many taxi days have FX rates
        with_fx = taxi_daily.join(
            fx, 
            taxi_daily.pickup_date == fx[f"{fx_date_col}"],
            "inner"
        ).count()
        
        total_taxi_days = taxi_daily.count()
        print(f"   Taxi days with FX rates: {with_fx}/{total_taxi_days} ({with_fx/total_taxi_days*100:.1f}%)")
        
except Exception as e:
    print(f"   ❌ Join error: {str(e)[:100]}")

# Test 3: Taxi Clean + Zones Join
print("\n🗺️ TEST 3: Taxi Clean + Zones Join")
try:
    taxi_clean = spark.table("silver_taxi_clean")
    zones = spark.table("dim_zones")
    
    # Test pickup zone join
    pickup_join = taxi_clean.join(
        zones,
        taxi_clean.pickup_zone == zones.zone_id,
        "left"
    ).limit(5)
    
    print(f"   Pickup zone join test:")
    pickup_join.select(
        "pickup_time",
        "pickup_zone",
        "zone_name",
        "zone_type"
    ).show()
    
except Exception as e:
    print(f"   ❌ Join error: {str(e)[:100]}")

# ============================================================================
# 5. DATA QUALITY ASSESSMENT
# ============================================================================
print("\n" + "="*80)
print("5️⃣ DATA QUALITY ASSESSMENT")
print("="*80)

print("""
📊 DATA QUALITY SUMMARY:

✅ STRENGTHS:
• Taxi data: 2.7M trips, good structure, Jan 2024 focus
• Air quality: Multiple pollutants, daily aggregates
• Dimensions: Date and zones available (514 zones)
• FX rates: Historical rates from 1999 to present
• Detailed taxi data: Hourly breakdown available

⚠️ LIMITATIONS:
• Limited date overlap between datasets
• Air quality data has gaps (some pollutants missing)
• No geographic coordinates for zones
• GDP data very limited (placeholder for USA only)

🎯 DATA COMPLETENESS:
1. Taxi Data: EXCELLENT (2.7M records, Jan 2024 focus)
2. Air Quality: GOOD but inconsistent (sparse measurements)
3. FX Rates: EXCELLENT (1999-present, daily rates)
4. Dimensions: GOOD (Date, Zones available)
5. GDP Data: LIMITED (placeholder only)

💡 KEY INSIGHTS:
• Strong foundation for taxi analytics
• Good environmental data for trend analysis
• FX data enables currency conversion
• Limited correlation potential but methodology can be demonstrated
""")

# ============================================================================
# 6. PRACTICAL RECOMMENDATIONS FOR GOLD LAYER
# ============================================================================
print("\n" + "="*80)
print("🎯 PRACTICAL RECOMMENDATIONS FOR GOLD LAYER")
print("="*80)

print("""
📊 WHAT YOU CAN REALISTICALLY ACHIEVE:

✅ DEFINITELY POSSIBLE:
1. Taxi mobility analysis (January 2024 focus)
   • Daily trip patterns and trends
   • Revenue analysis by day/hour  
   • Zone popularity analysis (514 zones)
   • Passenger count distribution
   • Trip duration analysis

2. Air quality trend analysis
   • Pollution patterns over available dates
   • Pollutant correlations (when data exists)
   • Daily/weekly patterns
   • Data quality handling demonstration

3. Economic integration
   • Convert USD revenue to EUR using FX rates
   • Demonstrate exchange rate impact
   • Show multi-currency reporting

⚠️ LIMITED BUT POSSIBLE:
4. Taxi vs Air Quality correlation
   • Limited overlapping days
   • Can demonstrate methodology
   • Show what WOULD be possible with better alignment
   • Create example visualizations

📌 GOLD LAYER STRATEGY:

FOCUS AREA 1: TAXI ANALYTICS (Strong Foundation)
  • Build comprehensive taxi fact tables
  • Create taxi-specific dimensions
  • Develop rich mobility dashboards

FOCUS AREA 2: ENVIRONMENTAL ANALYTICS (Good Data)  
  • Build pollution fact tables
  • Create pollutant dimensions
  • Demonstrate data quality handling

FOCUS AREA 3: ECONOMIC ANALYTICS (Excellent FX Data)
  • Currency conversion capabilities
  • Multi-currency reporting
  • Economic impact demonstration

FOCUS AREA 4: INTEGRATED ANALYSIS (Methodology Focus)
  • Demonstrate JOIN techniques
  • Show correlation methodology
  • Document real-world data challenges
""")

# ============================================================================
# 7. IMMEDIATE NEXT STEPS
# ============================================================================
print("\n" + "="*80)
print("🚀 IMMEDIATE NEXT STEPS")
print("="*80)

print("""
📌 CREATE GOLD LAYER WITH THESE PRIORITIES:

PHASE 1: BUILD FOUNDATION (Week 1)
1. FactTaxiDaily - Primary fact table
   • From silver_taxi_daily
   • Join with DimDate, DimZone
   • Add derived metrics
   
2. DimDateEnhanced - Enhanced date dimension
   • From dim_date_clean
   • Add fiscal periods, holidays, seasons
   • Add day characteristics
   
3. DimZoneEnhanced - Enhanced zone dimension
   • From dim_zones  
   • Add zone hierarchy if possible
   • Add geographic metadata placeholder

PHASE 2: ENVIRONMENTAL DATA (Week 1)
4. FactAirQualityDaily - Environmental fact table
   • From silver_air_quality_daily
   • Professional NULL handling
   • Data quality flags
   
5. DimPollutant - Pollutant dimension
   • PM2.5, NO2, O3, PM10 characteristics
   • Health impact categories
   • Regulatory thresholds

PHASE 3: ECONOMIC INTEGRATION (Week 2)  
6. DimCurrency - Currency dimension
   • From silver_fx_clean
   • USD/EUR exchange rates
   • Historical rate tracking
   
7. FactRevenueEUR - Currency converted revenue
   • Taxi revenue converted to EUR
   • Demonstrate currency impact
   • Multi-currency reporting

PHASE 4: ADVANCED ANALYTICS (Week 2)
8. FactTaxiHourly - Granular analysis
   • From silver_taxi_clean
   • Hourly patterns
   • Time-based analytics
   
9. BridgeTaxiAirQuality - Correlation bridge
   • For overlapping days
   • Correlation methodology
   • Example analysis

🎯 SUCCESS CRITERIA:
• Working star schema in Fabric Warehouse ✓
• Power BI dataset with all facts/dimensions ✓  
• 3 comprehensive dashboards ✓
• Complete documentation of data pipeline ✓
• Demonstration of real-world data challenges ✓
• Methodology for handling limited data ✓
""")

# ============================================================================
# 8. SUMMARY METRICS
# ============================================================================
print("\n" + "="*80)
print("📈 SUMMARY METRICS")
print("="*80)

# Calculate key metrics
try:
    # Taxi metrics
    taxi_daily = spark.table("silver_taxi_daily")
    total_taxi_days = taxi_daily.count()
    total_taxi_trips = taxi_daily.select(sum("trip_count")).collect()[0][0] or 0
    total_taxi_revenue = taxi_daily.select(sum("total_revenue_usd")).collect()[0][0] or 0
    
    # Air quality metrics
    air_daily = spark.table("silver_air_quality_daily")
    total_air_days = air_daily.count()
    air_days_with_pm25 = air_daily.filter(col("avg_pm25").isNotNull()).count()
    
    # Zone metrics
    zones = spark.table("dim_zones")
    total_zones = zones.count()
    pickup_zones = zones.filter(col("zone_type") == "Pickup").count()
    
    print(f"""
📊 KEY METRICS:
• Taxi Data: {total_taxi_days:,} days, {total_taxi_trips:,} trips, ${total_taxi_revenue:,.2f} revenue
• Air Quality: {total_air_days:,} days ({air_days_with_pm25:,} with PM2.5 data)
• Zones: {total_zones:,} total zones ({pickup_zones:,} pickup zones)
• FX Rates: 1999 to present (daily rates)
• Date Dimension: 2020-2024 ({spark.table('dim_date_clean').count():,} days)
""")
    
except Exception as e:
    print(f"Error calculating summary metrics: {str(e)[:100]}")

print(f"\n✅ Exploratory analysis completed: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("="*80)

StatementMeta(, bce670b9-bbdd-4176-af4c-daf4090bee40, 5, Finished, Available, Finished)

🔍 SILVER LAYER EXPLORATORY ANALYSIS
Analysis started: 2025-12-19 14:34:18

1️⃣ TABLE STRUCTURE ANALYSIS
Analyzing 11 active tables...

📊 silver_taxi_clean:
   Rows: 2,723,560
   Columns: 11
   First 5 columns: ['pickup_time', 'dropoff_time', 'passenger_count', 'distance_miles', 'pickup_zone']
   Date columns: ['pickup_time', 'dropoff_time', 'pickup_date']
   Date range: 2002-12-31 22:59:39 to 2024-02-01 00:01:15
   Numeric columns: 8 (e.g., ['passenger_count', 'distance_miles', 'pickup_zone'])
   Sample data:
-RECORD 0-------------------------------
 pickup_time      | 2024-01-24 15:17:12 
 dropoff_time     | 2024-01-24 15:34:53 
 passenger_count  | 1                   
 distance_miles   | 3.33                
 pickup_zone      | 239                 
 dropoff_zone     | 246                 
 fare_usd         | 20.5                
 total_usd        | 27.5                
 pickup_date      | 2024-01-24          
 pickup_hour      | 15                  
 duration_minutes | 17.7          